In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Vertex AI LLM 及 BigQuery 中的資料

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bigquery_ml_llm.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br>在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bigquery_ml_llm.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br>在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bigquery_ml_llm.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br>在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Rachael Deacon-smith](https://github.com/rachael-ds) |


## 概述

你可能會想知道：如何使用大型語言模型來處理儲存在資料倉庫中的資料？

在 [BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction) (BQML) 和 [Vertex AI LLMs](https://cloud.google.com/vertex-ai) (針對文字的 PaLM 2) 之間最新的整合中，這意味著組織現在可以在 BigQuery 資料上使用 Vertex AI LLMs。組織可以繼續使用 BigQuery 進行資料分析，同時也能利用生成式 AI 的強大功能，無需移動其資料。

在本教學課程中，你將逐步瞭解如何使用 Vertex AI LLMs 來處理儲存在 BigQuery 中的資料。


### 目標
目標是運用 BigQuery ML，展示 LLM 可以應用於 BigQuery 數據的多種方式。

你將執行呼叫 Vertex AI API 的簡單 SQL 語句，加上 `ML.GENERATE_TEXT` 函式，以：

- 摘要和分類文字
- 進行實體辨識
- 豐富資料
- 執行情緒分析


### 服務和費用
本教學課程使用以下 Google Cloud 資料分析和 ML 服務，它們是 Google Cloud 的計費元件：

* BigQuery 和 BigQuery ML <a href="https://cloud.google.com/bigquery/pricing" target="_blank">(定價)</a>
* Vertex AI API <a href="https://cloud.google.com/vertex-ai/pricing" target="_blank">(定價)</a>

查看 [BQML 定價頁面](https://cloud.google.com/bigquery/pricing#bqml)以了解如何計算這些服務的費用。

使用 [定價計算器](https://cloud.google.com/products/calculator)根據預計使用量產生預算估計。


### 安裝

安裝執行此筆記本所需的下列套件。


In [ ]:
%pip install --upgrade --user google-cloud-bigquery-connection google-cloud-aiplatform

#### 重新啟動當前執行時期

要在這個 Jupyter 執行時期中使用新安裝的套件，你必須重新啟動執行時期。你可以執行以下Cell來重新啟動當前核心，就可以做到這一點。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 開始前，請等待 Kernel 重啟完畢 ⚠️</b>
</div>


#### 設定專案和 Bigquery 區域

在建立 BigQuery 資料集和雲端資源連線時，你需要設定 `PROJECT_ID` 和 `REGION` 變數。

目前，只有 `us` 多區域和 `us-central1` 單一區域支援 BigQuery 的遠端模型服務。

**對於這個筆記本，將區域設定為 `US` 以確保存取下方使用的所有公開資料集。** 

進一步了解[BigQuery 公開資料集區域](https://cloud.google.com/bigquery/public-data?gad=1&gclid=CjwKCAjw_aemBhBLEiwAT98FMhtM2q0Il2M4xU_eLwO_mAJpaZuuzBlQCNEkHKDDI-snZyGguxqnaRoCBdYQAvD_BwE&gclsrc=aw.ds#public_dataset_locations)。


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "US"  # @param {type: "string"}

#### 設定專案變數


這些變數將在整個筆記本中使用


*   **DATASET_ID:** BigQuery 資料集的 ID
*   **CONN_NAME** : 用於連線至 Vertex AI 服務的 BigQuery 連結器名稱
*   **LLM_MODEL_NAME** : 在 BigQuery 中建立的 LLM 賦予的名稱


In [ ]:
DATASET_ID = "bqml_llm"
CONN_NAME = "bqml_llm_conn"
LLM_MODEL_NAME = "bqml-vertex-llm"

### 驗證你的筆記本環境
如果你使用 **Colab** ，你將需要先執行驗證。下一個Cell將會檢查你目前是否使用 Colab，並將開始驗證程序。

如果你使用 **Vertex AI Workbench** ，你不需要其他驗證。如需更多資訊，你可以查看 [這裡](https://github.com/doggy8088/generative-ai/tree/main/setup-env) 的設定說明。


In [ ]:
import sys

# Addtional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### 匯入函式庫


In [ ]:
from google.cloud import bigquery
from google.cloud import bigquery_connection_v1 as bq_connection
import pandas as pd

pd.set_option("display.max_colwidth", 1000)

### 建立 BigQuery Cloud 資源連線

你需要建立 [Cloud 資源連線](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) 以讓 BigQuery 能與 Vertex AI 服務互動。

你可能需要先 [啟用 BigQuery 連線 API](https://console.developers.google.com/apis/api/bigqueryconnection.googleapis.com/overview)。


In [ ]:
client = bq_connection.ConnectionServiceClient()
new_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}"
exists_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}/connections/{CONN_NAME}"
cloud_resource_properties = bq_connection.CloudResourceProperties({})

# Try to use an existing connection if one already exists. If not, create a new one.
try:
    request = client.get_connection(
        request=bq_connection.GetConnectionRequest(name=exists_conn_parent)
    )
    CONN_SERVICE_ACCOUNT = f"serviceAccount:{request.cloud_resource.service_account_id}"
except Exception:
    connection = bq_connection.types.Connection(
        {"friendly_name": CONN_NAME, "cloud_resource": cloud_resource_properties}
    )
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": new_conn_parent,
            "connection_id": CONN_NAME,
            "connection": connection,
        }
    )
    response = client.create_connection(request)
    CONN_SERVICE_ACCOUNT = (
        f"serviceAccount:{response.cloud_resource.service_account_id}"
    )
print(CONN_SERVICE_ACCOUNT)

### 設定服務帳戶的權限
資源連接服務帳戶需要某些專案層級權限，這些權限於 <a href="https://cloud.google.com/bigquery/docs/bigquery-ml-remote-model-tutorial#set_up_access" target="_blank">Vertex AI 函式文件</a> 中列出。

<br>

**注意：** 如果你使用 **Vertex AI Workbench** ，Vertex AI 使用的服務帳戶可能沒有足夠的權限來新增 IAM 政策繫結。你可能會看到錯誤：
> `錯誤：(gcloud.projects.add-iam-policy-binding) 使用者 [12345-compute@developer.gserviceaccount.com] 沒有權限存取專案執行個體 [my-project-id:setIamPolicy] (或它可能不存在)：拒絕政策更新存取權。`

如果你在 Vertex AI Workbench 中看到上述錯誤，請開啟終端機 (檔案 -> 新建 -> 終端機)，然後透過 `gcloud auth login` 驗證你的 GCP 使用者帳戶並遵循其中的步驟。驗證後，返回這個筆記本並執行下方的Cell。或者，你可以設定 IAM 角色


In [ ]:
gcloud_serviceusage = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{CONN_SERVICE_ACCOUNT}" --role="roles/serviceusage.serviceUsageConsumer"
"""

gcloud_bigquery = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{CONN_SERVICE_ACCOUNT}" --role="roles/bigquery.connectionUser"
"""

gcloud_aiplatform = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{CONN_SERVICE_ACCOUNT}" --role="roles/aiplatform.user"
"""

print(gcloud_serviceusage)
!$gcloud_serviceusage #execute gcloud script

print(gcloud_bigquery)
!$gcloud_bigquery #execute gcloud script

print(gcloud_aiplatform)
!$gcloud_aiplatform #execute gcloud script

你可以執行下方Cell來確認已設定這三個 IAM 角色：
- roles/aiplatform.user
- roles/bigquery.connectionUser
- roles/serviceusage.serviceUsageConsumer


In [ ]:
!gcloud projects get-iam-policy $PROJECT_ID  \
--flatten="bindings[].members" \
--format="table(bindings.role)" \
--filter="bindings.members:$CONN_SERVICE_ACCOUNT"

## 準備 BigQuery 資料集


### 建立 BigQuery 資料集
你將需要一個 BigQuery 資料集來儲存你的機器學習模型和表格。此資料集必須在 BigQuery Cloud 資源連接所使用的相同區域中建立。

執行以下動作以在你的專案內建立資料集：


In [ ]:
client = bigquery.Client(project=PROJECT_ID)

dataset_id = f"""{PROJECT_ID}.{DATASET_ID}"""
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION

dataset = client.create_dataset(dataset, exists_ok=True)

print(f"Dataset {dataset.dataset_id} created.")

建立Wrapper以使用BigQuery客戶端執行查詢並回傳結果：


In [ ]:
# Wrapper to use BigQuery client to run query and return result


def run_bq_query(sql: str):
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results or error, if any
    """
    try:
        query_job = client.query(sql)
        result = query_job.result()
        print(f"JOB ID: {query_job.job_id} STATUS: {query_job.state}")
        return result

    except Exception as e:
        raise Exception(str(e))

## 使用大型語言模型和 BigQuery ML


要與 BigQuery ML 一起使用 LLM，你首先需要設定 LLM，然後執行帶有提示的 `ML.GENERATE_TEXT` 函式。這一切都可以用 SQL 完成。


### 設定 Vertex AI 模型

你可以在 BigQuery 使用 CREATE MODEL 語句來設定 Vertex AI 遠端模型：


In [ ]:
sql = f"""
      CREATE OR REPLACE MODEL
        `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`
        REMOTE WITH CONNECTION
          `{PROJECT_ID}.{REGION}.{CONN_NAME}`
          OPTIONS ( remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1');
      """
result = run_bq_query(sql)

### 使用 LLM
你可以對大查詢資料中儲存的資料或自由文字執行 `ML.GENERATE_TEXT` 函式，以在 BQML 中使用 LLM

[BQML document](https://cloud.google.com/bigquery/docs/generate-text#generate_text) 提供了關於使用參數的更多詳細資訊：`temperature, max_output_tokens, top_p and top_k.`

*請注意：輸入文字的資料表欄位必須有別名 'prompt'*


In [ ]:
PROMPT = "Describe a cat in one paragraph"

sql = f"""
          SELECT
            *
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                '{PROMPT}' AS prompt
              ),
              STRUCT
              (
                1 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output
              ));
        """
result = run_bq_query(sql)
result.to_dataframe()

在這種情況下，LLM 對描述一隻貓的簡單提示做出了回應，只用了一段。

LLM 的回應會以 BigQuery 中的結果表格形式返回。表格包括可解析為提取內容結果的 JSON。

將 `flatten_json_output` 參數設定為 TRUE 將返回一個展平的 JSON，作為一個字串：`ml_generate_text_llm_result`。

對於其餘範例，為了簡化起見，你可以只顯示提示和 `ml_generate_text_llm_result`。


## 範例使用案例

以下範例探討使用 BQML LLM 執行內容創作、文字摘要、分類、實體辨識、資料豐富化及情緒分析。

撰寫你自己的提示時，我們建議你先檢閱這些 [提示設計最佳實務](https://github.com/doggy8088/generative-ai/blob/main/language/intro_prompt_design.ipynb)。


#### 文本分類

本範例將新聞文章分類成以下類別之一：科技、體育、商業、政治或娛樂。這些文章儲存在 BigQuery BBC News 公共資料集。


In [ ]:
PROMPT = "Please categorize this BBC news article into either tech, sport, business, politics, or entertainment and return the category. Here is an example. News article: Intel has unveiled research that could mean data is soon being moved around chips at the speed of light., Category: Tech "

sql = f"""
          SELECT
            body AS article_body,
            CONCAT('{PROMPT}','News article: ', 'article_body', ', Category:') as prompt_template,
            ml_generate_text_llm_result as llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT}','News article: ', body, ', Category:') AS prompt,
                body
              FROM
                `bigquery-public-data.bbc_news.fulltext`
              LIMIT
                5),
              STRUCT(1 AS temperature, 1024 AS max_output_tokens, 0.8 AS top_p, 40 AS top_k, TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)
result.to_dataframe()

#### 文字摘要

這個範例將儲存在 BigQuery BBC 新聞公開資料集中的新聞文章改寫成 12 歲的兒童也可理解的文字。


In [ ]:
PROMPT = "Please rewrite this article to enable easier understanding for a 12 year old. Article: "

sql = f"""
          SELECT
            body as article_before,
            CONCAT('{PROMPT}', 'article_before') as prompt_template,
            ml_generate_text_llm_result as llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT}', body) AS prompt,
                body
              FROM  `bigquery-public-data.bbc_news.fulltext`
              LIMIT 5
              ),
              STRUCT(1 AS temperature, 1024 AS max_output_tokens, 0.8 AS top_p, 40 AS top_k, TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)
result.to_dataframe()

此範例將 BigQuery BBC News 公共資料集內儲存的冗長新聞文章摘要為 25 個字或更少


In [ ]:
PROMPT = "Please summarize this BBC news article into 25 words or less: "

sql = f"""
          SELECT
            body as article_before,
            ARRAY_LENGTH(SPLIT(body, ' ')) AS word_count_before,
            CONCAT('{PROMPT}') as prompt_template,
            ml_generate_text_llm_result as article_after,
            ARRAY_LENGTH(SPLIT(ml_generate_text_llm_result, ' ')) AS word_count_after,
            1-ARRAY_LENGTH(SPLIT(ml_generate_text_llm_result, ' '))/ARRAY_LENGTH(SPLIT(body, ' ')) as percent_reduction_words
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT}', body) AS prompt,
                body
              FROM
                `bigquery-public-data.bbc_news.fulltext`
              LIMIT
                5),
              STRUCT(1 AS temperature, 1024 AS max_output_tokens, 0.8 AS top_p, 40 AS top_k, TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)
result.to_dataframe()

結果字數不一定都會在要求的 25 字以內，因此可能需要採取進一步的 [提示工程](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/introduction-prompt-design)。


#### 實體辨識

這個例子提取含有統計資料的新聞文章中的句子。這些文章儲存在 BigQuery BBC News 公開資料集中。


In [ ]:
PROMPT = "Please return a bullet-point list of all sentences in this article that cite a statistic: "

sql = f"""
          SELECT
            body AS article_body,
            CONCAT('{PROMPT}', 'article_body') AS prompt,
            ml_generate_text_llm_result AS llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT}', body) AS prompt,
                body
              FROM
                `bigquery-public-data.bbc_news.fulltext`
              LIMIT
                5),
              STRUCT(1 AS temperature, 1024 AS max_output_tokens, 0.8 AS top_p, 40 AS top_k, TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)
result.to_dataframe()

此範例從儲存在 BigQuery thelook_ecommerce 公共資料集中的產品描述中，萃取品牌名稱。


In [ ]:
PROMPT = "Please return the brand name listed in this product description. Here is an example. Product: TYR Sport Mens Solid Durafast Jammer Swim Suit, Brand: TYR ; Product: "

sql = f"""
          SELECT
            name AS product_description,
            CONCAT('{PROMPT}', 'product_description,',' Brand: ') as prompt_template,
            ml_generate_text_llm_result as llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT}', name,' Brand: ') AS prompt,
                name
              FROM
                `bigquery-public-data.thelook_ecommerce.products`
              LIMIT
                5),
              STRUCT(1 AS temperature, 1024 AS max_output_tokens, 0.8 AS top_p, 40 AS top_k, TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)
result.to_dataframe()

#### 情緒分析

這項範例會對儲存在 BigQuery IMDB 公用資料集的電影評論執行情緒分析，以判斷電影評論是正向、負向或中立。


In [ ]:
PROMPT = "Please categorize this movie review as either Positive, Negative or Neutral. Here is an example. Review: I dislike this movie, Sentiment: Negative"

sql = f"""
          SELECT
            review,
            CONCAT('{PROMPT}',' Review: review Sentiment:') as prompt_template,
            ml_generate_text_llm_result as llm_result

          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT}',' Review: ', review, ', Sentiment:') AS prompt,
                review
              FROM
                `bigquery-public-data.imdb.reviews`
              WHERE
                UPPER(title) = 'TROY'
              LIMIT
                10 ),
              STRUCT(0.2 AS temperature,
                50 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k, TRUE AS flatten_json_output))
          """
result = run_bq_query(sql)
result.to_dataframe()

#### 內容創作


本範例根據收件者人口統計和支出資料建立一個行銷活動電子郵件。商業資料從 BigQuery 的 [thelook 電子商務公眾資料集](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce) 取得。


首先，你需要將資料集的 order_items、products 和 users 表格加入，才能取得包含電子郵件資訊的表格，其中包括購買的品項、該品項的描述，以及購買者的統計資料。


In [ ]:
sql = f"""
      CREATE OR REPLACE TABLE
        `{PROJECT_ID}.{DATASET_ID}.purchases` AS
      SELECT
        u.id,
        u.first_name,
        u.email,
        u.postal_code,
        u.country,
        o.order_id,
        o.created_at,
        p.category,
        p.name
      FROM
        `bigquery-public-data.thelook_ecommerce.users` u
      JOIN (
        SELECT
          user_id,
          order_id,
          created_at,
          product_id,
          ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at DESC) AS rn
        FROM
          `bigquery-public-data.thelook_ecommerce.order_items`
      ) o
      ON
        u.id = o.user_id
      JOIN
        `bigquery-public-data.thelook_ecommerce.products` p
      ON
        o.product_id = p.id
      WHERE
        o.rn = 1 AND p.category = "Active" AND u.country = "United States";
       """

result = run_bq_query(sql)

查詢新表格，你將會看到每筆購買的綜合資料集。


In [ ]:
sql = f"""
        SELECT
            *
        FROM
            `{PROJECT_ID}.{DATASET_ID}.purchases`
        LIMIT
            10;
      """
result = run_bq_query(sql)
result.to_dataframe().head(10)

現在你會為 LLM 準備提示，並納入商品名稱、購買者的姓名和郵遞區號。


In [ ]:
PROMPT_PART1 = "A user bought a product with this description: "
PROMPT_PART2 = ' Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example "We hope you are enjoying your new t-shirt". '
PROMPT_PART3 = "Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. "
PROMPT_PART4 = "Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code "
PROMPT_PART5 = '. Do not mention the brand of the product, just sign off the email with "-TheLook." Address the email to: '

sql = f"""
          SELECT
            prompt,
            ml_generate_text_llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT_PART1}',name,'{PROMPT_PART2}','{PROMPT_PART3}','{PROMPT_PART4}',postal_code,'{PROMPT_PART5}',first_name) AS prompt
              FROM
                `{PROJECT_ID}.{DATASET_ID}.purchases`
              LIMIT
                5),
              STRUCT(1 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)
result.to_dataframe()

## 清除
如需清除本專案中所用的所有 Google Cloud 資源，你可 <a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">刪除 Google Cloud
專案</a>，你已對此教學課程使用其。

否則，你可以透過取消註解下方內容，刪除你在本教學課程中建立的個別資源：


In [ ]:
# # Delete BigQuery dataset, including the BigQuery ML models you just created, and the BigQuery Connection
# ! bq rm -r -f $PROJECT_ID:$DATASET_ID
# ! bq rm --connection --project_id=$PROJECT_ID --location=$REGION $CONN_NAME

**總結** 

在本教學中，我們展示如何整合 BQML 與 Vertex AI LLM 並提供範例說明如何將新的 `ML.GENERATE_TEXT` 函式直接應用到儲存在 BigQuery 中的資料。

查看我們的 [BigQuery ML LLM 網頁](https://cloud.google.com/bigquery/docs/inference-overview#generative_ai) 以進一步了解 BigQuery 中的遠端模型和生成式 AI。
